In [1]:
import pandas as pd
import os
from tqdm.notebook import tqdm

In [4]:
file = open('mobiact_dataset/Readme.txt', 'r', encoding='latin1')
strings = file.readlines()
file.close()
for s in strings:
    print(s)


===========================MobiActDataset===========================

The NEW version of the MobiAct dataset includes:

 	Four different types of falls performed by 66 participants

 	Eleven different types of ADLs performed by 19 participants and nine types of ADLs performed by 59 participants (plus one activity "LYI" which results from the inactivity period after a fall by 66 participants)

 	Five sub-scenarios which construct one scenario of daily living, which consists of a sequence of 50 activities and performed by 19 participants.



The new released version of the MobiAct dataset includes:

  The raw recorded data in txt format, separated by each activity

  The annotated data in csv format, separated by each activity





Filename format:

<ADL OR FALL OR SCENARIO_CODE>_<SENSOR_CODE>_<SUBJECT_ID>_<TRIAL_NO>.txt



examples:

1 -->	WAL_acc_5_1.txt

2 -->	STD_ori_9_5.txt

3 -->	FKL_gyro_3_2.txt

4 -->	SRH_acc_1_1.txt





Subjects:

+------+---------+-----------+-------+---

In [5]:
person_list = []
for s in strings:
    if 'sub' in s and '|' in s:
        temp = s.split('|')
        temp = [x.strip() for x in temp]
        if len(temp) == 9:
            person_list.append(temp[3:-1])

activity_list = []
for s in strings:
    if '|' in s:
        temp = s.split('|')
        temp = [x.strip() for x in temp]
        if len(temp) == 8:
            activity_list.append(temp[1:-1])

In [6]:
falls = ['FOL', 'FKL', 'BSC', 'SDL']

columns = ['name', 'age', 'height', 'weight', 'gender']
person_info = pd.DataFrame(person_list, columns=columns)

activity_info = pd.DataFrame(activity_list)
activity_info.columns = activity_info.iloc[0]
activity_info = activity_info.drop(0)
activity_info = activity_info.drop(13)
activity_info = activity_info.reset_index(drop=True)
# print(activity_info.columns)
index = activity_info['No.']
activity_info = activity_info.drop(['No.'], axis=1)
activity_info.index = index
activity_info['label_encoded'] = list(range(len(activity_info)))

In [8]:
activity_info.to_csv('mobiact_preprocessed/activity_info.csv')

In [ ]:
data_dir = 'MobiAct_Dataset_v2.0/Annotated Data/'
act_list = os.listdir(data_dir)
print(act_list)

In [ ]:
train = set()
valid = set()
test = set()
for act in tqdm(act_list):
    file_list = os.listdir(data_dir + act + '/')
    for file in file_list:
        data = pd.read_csv(data_dir + act + '/' +file)
        file_names = file.split('_')
        person_num = int(file_names[1])
        person_id = person_num - 1
        trial_num = file_names[2]
        
        if person_num <= 50:
            train.add(person_num)
        elif person_num > 50 and person_num <= 53:
            valid.add(person_num)
        else:
            test.add(person_num)

In [ ]:
train.to_csv('mobiact_preprocessed/train.csv')
valid.to_csv('mobiact_preprocessed/valid.csv')
test.to_csv('mobiact_preprocessed/test.csv')